In [68]:
import requests
from bs4 import BeautifulSoup
import pprint
import json
import time
import re

In [20]:
def download_all_html():
    """
    下载所有页面
    """
    htmls = []
#     for idx in range(0,5856,20):
    for idx in range(0,200,20):
        url = f"https://movie.douban.com/subject/26931786/comments?start={idx}&limit=20"
        print("craw html:",url)
        r = requests.get(url)
        if r.status_code !=200:
            raise Exception("Error")
        htmls.append(r.text)
        time.sleep(1)
    return htmls

In [21]:
htmls = download_all_html()

craw html: https://movie.douban.com/subject/26931786/comments?start=0&limit=20
craw html: https://movie.douban.com/subject/26931786/comments?start=20&limit=20
craw html: https://movie.douban.com/subject/26931786/comments?start=40&limit=20
craw html: https://movie.douban.com/subject/26931786/comments?start=60&limit=20
craw html: https://movie.douban.com/subject/26931786/comments?start=80&limit=20
craw html: https://movie.douban.com/subject/26931786/comments?start=100&limit=20
craw html: https://movie.douban.com/subject/26931786/comments?start=120&limit=20
craw html: https://movie.douban.com/subject/26931786/comments?start=140&limit=20
craw html: https://movie.douban.com/subject/26931786/comments?start=160&limit=20
craw html: https://movie.douban.com/subject/26931786/comments?start=180&limit=20


In [24]:
htmls[0]

'<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <meta name="referrer" content="always">\n    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />\n    <title>\n\n蜘蛛侠：英雄远征 短评\n</title>\n    \n    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />\n    <meta http-equiv="Pragma" content="no-cache">\n    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">\n    \n    <meta name="keywords" content="蜘蛛侠：英雄远征,Spider-Man: Far from Home,影讯,排片,放映时间,电影票价,在线购票"/>\n    <meta name="description" content="蜘蛛侠：英雄远征短评" />\n    <meta name="mobile-agent" content="format=html5; url=http://m.douban.com/movie/subject/26931786/comments"/>\n    <script type="text/javascript" src="https://img3.doubanio.com/f/shire/77323ae72a612bba8b65f845491513ff3329b1bb/js/do.js" data-cfg-autoload="false"></script>\n\n    

In [71]:

def parse_single_html(html):
    """
解析豆瓣评论，返回数据
@return list({"starts","date","comment","user","support"})
    """
    soup = BeautifulSoup(html,'html.parser')
    articles = soup.find("div",class_ = "article").find("div",class_="mod-bd").find_all("div",class_="comment-item")
    datas = []
#     print(articles)
    for article in articles:
        user_note = (
            article
            .find("div",class_="avatar")
            .find("a")
        )
        user = user_note["title"]
        start_note = (
            article
            .find("div",class_="comment")
            .find("h3").find("span",class_="comment-info").find_all("span")
        )
#         print(start_note[1]['class'][0])
        start = start_note[1]['class'][0] + start_note[1]['title']
        start = re.sub('[，。、a-zA-Z（）]','',start)
        date_note = (
            article
            .find("div",class_="comment")
            .find("h3").find("span",class_="comment-info").find("span",class_="comment-time")
        )
        date = date_note.get_text().strip()
        support_note = (
            article
            .find("div",class_="comment")
            .find("h3").find("span",class_="comment-vote").find("span",class_="votes")
        )
        support = support_note.get_text()
        comment_note = (
            article
            .find("div",class_="comment")
            .find("p").find("span",class_="short")
        )
        comment = comment_note.get_text()
        datas.append(
            {"user":user,"start":start,"date":date,"support":support,"comment":comment}
        )
    return datas
        

In [72]:
pprint.pprint(parse_single_html(htmls[0]))

[{'comment': '当看到前30分钟的时候，我猜测这部应该没啥看点又要学上一部归来讲小屁孩欧洲旅游顺便谈恋爱最后一把鸡汤责任越大能力越大的那点破事，结果剧情就成功反转奥创纪元2.0。当看了电影前58分钟我还在怀疑Marvel '
             '这部电影编剧强行划水，从环太平洋搬来几个怪兽模型搞个烂怂反派拍续集，然后被编剧狠狠打了脸。当看到电影提到神秘客说称自己是来自平行宇宙的超级英雄时，我还在思考会不会是为下一部新solo客串加铺垫时候，然后编剧狠狠嘲讽了平行世界脑洞。最后彩蛋，当出字幕时思考为啥尼克之前说过不要再提惊奇队长的话是什么意思时候，彩蛋解释原来是斯克鲁人。非常精彩的多次反套路出牌，Marvel '
             '再次用这部电影证明了自己在把控故事结构和气氛高潮到了炉火纯青的境界，同期上映的那部福克斯的简直就是丢死人了！',
  'date': '2019-06-28',
  'start': '40推荐',
  'support': '5631',
  'user': 'Ada🧣Wang💋💍'},
 {'comment': '特效达到了一个新的高度。片中加彩蛋双反转防不胜防，电影更加趋向理性化，将能力不可控的矛盾重新提出，与身份性合成更大的矛盾。',
  'date': '2019-06-28',
  'start': '40推荐',
  'support': '3411',
  'user': '林萧路寒'},
 {'comment': '当哈皮看着帕克做战衣的时候，或许他想的和我们想的都一样：想你了，托尼。',
  'date': '2019-06-28',
  'start': '50力荐',
  'support': '6714',
  'user': '唐一'},
 {'comment': '不是亿万富翁，不是天神后代，之所以成为漫威一哥，就是因为他最接近普通人。\n'
             '他并不完美，相反，他会犯错，会轻易相信别人；\n'
             '会自卑，会觉得自己的能力配不上梦想。\n'
             '但正是这些缺点让他更加有血有肉，更让人感同身受。\n'
             '他也想和MJ一起看话剧，但不得不去拯救世界；\n'
             '

In [73]:
all_datas = []
for html in htmls:
    all_datas.extend(parse_single_html(htmls[0]))
    

In [74]:
all_datas

[{'user': 'Ada🧣Wang💋💍',
  'start': '40推荐',
  'date': '2019-06-28',
  'support': '5631',
  'comment': '当看到前30分钟的时候，我猜测这部应该没啥看点又要学上一部归来讲小屁孩欧洲旅游顺便谈恋爱最后一把鸡汤责任越大能力越大的那点破事，结果剧情就成功反转奥创纪元2.0。当看了电影前58分钟我还在怀疑Marvel 这部电影编剧强行划水，从环太平洋搬来几个怪兽模型搞个烂怂反派拍续集，然后被编剧狠狠打了脸。当看到电影提到神秘客说称自己是来自平行宇宙的超级英雄时，我还在思考会不会是为下一部新solo客串加铺垫时候，然后编剧狠狠嘲讽了平行世界脑洞。最后彩蛋，当出字幕时思考为啥尼克之前说过不要再提惊奇队长的话是什么意思时候，彩蛋解释原来是斯克鲁人。非常精彩的多次反套路出牌，Marvel 再次用这部电影证明了自己在把控故事结构和气氛高潮到了炉火纯青的境界，同期上映的那部福克斯的简直就是丢死人了！'},
 {'user': '林萧路寒',
  'start': '40推荐',
  'date': '2019-06-28',
  'support': '3411',
  'comment': '特效达到了一个新的高度。片中加彩蛋双反转防不胜防，电影更加趋向理性化，将能力不可控的矛盾重新提出，与身份性合成更大的矛盾。'},
 {'user': '唐一',
  'start': '50力荐',
  'date': '2019-06-28',
  'support': '6714',
  'comment': '当哈皮看着帕克做战衣的时候，或许他想的和我们想的都一样：想你了，托尼。'},
 {'user': '凌睿',
  'start': '50力荐',
  'date': '2019-06-28',
  'support': '4083',
  'comment': '不是亿万富翁，不是天神后代，之所以成为漫威一哥，就是因为他最接近普通人。\n他并不完美，相反，他会犯错，会轻易相信别人；\n会自卑，会觉得自己的能力配不上梦想。\n但正是这些缺点让他更加有血有肉，更让人感同身受。\n他也想和MJ一起看话剧，但不得不去拯救世界；\n他也想追求喜欢的女生，但他选择了牺牲小我，成就大我。\n因为“能

In [75]:
with open("all_datas.json","w") as fout:
    for data in all_datas:
        fout.write(json.dumps(data,ensure_ascii=False)+"\n")